In [88]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import email
import email.policy # 使用推荐的策略来解析
from bs4 import BeautifulSoup
import os # 用于检查文件是否存在
import pandas as pd

user_data_dir = r"C:\Users\wangz\AppData\Local\Google\Chrome\User Data" # <-- 把 '你的用户名' 换成你的实际 Windows 用户名
profile_directory = "Default"  # 或者 "Profile 1", "Profile 2" 等，取决于 chrome://version 显示的

target_class = 'product__message-title_area' # 你要查找的 class 名称
html_content = None # 用来存储提取出来的 HTML 文本
# -----------
excel_file_name = 'coles_data.xlsx'
product_data = []

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument(f"user-data-dir={user_data_dir}")
chrome_options.add_argument(f"profile-directory={profile_directory}")

chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument("--disable-extensions") # 注意：这个选项可能会阻止加载你本地配置中的扩展，如果需要扩展运行，可以注释掉这行
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = None # 初始化 driver 变量
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.set_window_size(1920, 1080)
    main_page_url = "https://www.coles.com.au"
    driver.get(main_page_url)
    time.sleep(5)

    pageNumber = 2
    special_url = f'https://www.coles.com.au/on-special?filter_Special=halfprice&page={pageNumber}'
    driver.get(special_url)
    time.sleep(5)
    result = driver.execute_cdp_cmd('Page.captureSnapshot', {'format': 'mhtml'})
    msg = email.message_from_string(result['data'])
    for part in msg.walk():
        content_type = part.get_content_type()
        # 主要的HTML内容通常是 'text/html'
        if content_type == 'text/html':
            # 获取内容负载并解码 (decode=True 会处理 base64 或 quoted-printable 编码)
            payload_bytes = part.get_payload(decode=True)
            # 尝试从 header 获取编码，否则默认 utf-8
            charset = part.get_content_charset() or 'utf-8'
            try:
                # 将字节解码为字符串
                html_content = payload_bytes.decode(charset)
                print(f"成功提取并解码 HTML 内容 (使用编码: {charset})。")
                break # 找到主要的 HTML 部分就停止
            except UnicodeDecodeError:
                print(f"警告：使用声明的编码 '{charset}' 解码失败，尝试使用 'utf-8' 作为备用...")
                try:
                    html_content = payload_bytes.decode('utf-8')
                    print("成功使用 'utf-8' 备用编码解码。")
                    break
                except Exception as decode_err:
                    print(f"使用备用编码解码也失败: {decode_err}")
                    # 这里可以选择继续查找其他 text/html 部分，或者直接放弃
                    html_content = None # 重置以防部分解码
                    continue # 继续查找下一个部分

    # 4. 如果成功提取了 HTML 内容，则使用 BeautifulSoup 解析
    if html_content:
        print(f"\n正在使用 BeautifulSoup 解析提取的 HTML...")
        soup = BeautifulSoup(html_content, 'html.parser')

        print(f"查找所有 class 为 '{target_class}' 的 div 元素...")
        # 5. 查找所有符合条件的 div 元素
        elements = soup.find_all('div', class_=target_class)

        # 6. 打印找到的元素内容
        if elements:
            print(f"找到了 {len(elements)} 个匹配的元素：")
            for i, element in enumerate(elements):
                product_info = {
                    '产品名称': "N/A",
                    '原价': "N/A",
                    '现价': "N/A",
                    '单位价格': "N/A"
                }
                product_info['产品名称'] = element.find("h2", class_="product__title").contents[0]
                product_info['原价'] = element.find("span", class_="price__was").contents[0].replace(" | Was ", "")
                product_info['现价'] = element.find("span", class_="price__value").contents[0]
                product_info['单位价格'] = element.find("div", class_="price__calculation_method").contents[0]
                product_data.append(product_info)
        else:
            print(f"在 HTML 内容中未找到任何 class 为 '{target_class}' 的 div 元素。")
    else:
        print("错误：未能在 MHTML 文件中找到有效的 HTML 内容部分。")
except Exception as e:
    print(f"在处理页面或保存 MHTML 时出错: {e}")
    # import traceback; traceback.print_exc() # 取消注释查看详细错误栈

finally:
    if driver:
        print("正在关闭 WebDriver...")
        driver.quit()
        print("WebDriver 已关闭。")
    else:
        print("WebDriver 未成功初始化，无需关闭。")

if product_data:
    print(f"\n正在将提取的 {len(product_data)} 条产品数据保存到 Excel 文件: {excel_file_name}")
    try:
        # 将字典列表转换为 pandas DataFrame
        df = pd.DataFrame(product_data)
        # 可以指定列的顺序
        df = df[['产品名称', '现价', '原价', '单位价格']]
        # 将 DataFrame 写入 Excel 文件，不包含索引列
        df.to_excel(excel_file_name, index=False, engine='openpyxl')
        print(f"Excel 文件 '{excel_file_name}' 保存成功。")
    except ImportError:
            print("错误: 需要安装 'pandas' 和 'openpyxl' 库来保存 Excel 文件。请运行: pip install pandas openpyxl")
    except Exception as ex:
            print(f"保存 Excel 文件时出错: {ex}")
else:
    print("没有提取到任何产品数据，未创建 Excel 文件。")

成功提取并解码 HTML 内容 (使用编码: utf-8)。

正在使用 BeautifulSoup 解析提取的 HTML...
查找所有 class 为 'product__message-title_area' 的 div 元素...
找到了 48 个匹配的元素：
正在关闭 WebDriver...
WebDriver 已关闭。

正在将提取的 48 条产品数据保存到 Excel 文件: coles_data.xlsx
Excel 文件 'coles_data.xlsx' 保存成功。


In [100]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import email
from bs4 import BeautifulSoup
import pandas as pd

user_data_dir = r"C:\Users\wangz\AppData\Local\Google\Chrome\User Data"
profile_directory = "Default"

target_class = 'product__message-title_area'
product_data = []
excel_file_name = 'coles_data.xlsx'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-data-dir={user_data_dir}")
chrome_options.add_argument(f"profile-directory={profile_directory}")
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = None
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.set_window_size(1920, 1080)
    main_page_url = "https://www.coles.com.au"
    driver.get(main_page_url)
    time.sleep(20)

    special_url_base = 'https://www.coles.com.au/on-special?filter_Special=halfprice&page='
    current_page = 1
    total_pages = 1  # 初始化为 1

    while current_page <= total_pages:
        special_url = f'{special_url_base}{current_page}'
        driver.get(special_url)
        time.sleep(5)
        result = driver.execute_cdp_cmd('Page.captureSnapshot', {'format': 'mhtml'})
        msg = email.message_from_string(result['data'])
        html_content = None
        for part in msg.walk():
            content_type = part.get_content_type()
            if content_type == 'text/html':
                payload_bytes = part.get_payload(decode=True)
                charset = part.get_content_charset() or 'utf-8'
                try:
                    html_content = payload_bytes.decode(charset)
                    print(f"成功提取并解码第 {current_page} 页的 HTML 内容 (使用编码: {charset})。")
                    break
                except UnicodeDecodeError:
                    print(f"警告：使用声明的编码 '{charset}' 解码第 {current_page} 页失败，尝试使用 'utf-8' 作为备用...")
                    try:
                        html_content = payload_bytes.decode('utf-8')
                        print(f"成功使用 'utf-8' 备用编码解码第 {current_page} 页。")
                        break
                    except Exception as decode_err:
                        print(f"使用备用编码解码第 {current_page} 页也失败: {decode_err}")
                        html_content = None
                        continue
                except Exception as decode_err:
                    print(f"解码第 {current_page} 页时发生其他错误: {decode_err}")
                    html_content = None
                    continue

        if html_content:
            soup = BeautifulSoup(html_content, 'html.parser')

            # 在第一页获取总页数
            if current_page == 1:
                page_buttons = soup.find_all("span", class_="MuiButtonBase-root")
                if page_buttons:
                    try:
                        total_pages_text = page_buttons[-1].get_text(strip=True)
                        total_pages = int(total_pages_text)
                        print(f"总页数已找到: {total_pages}")
                    except ValueError:
                        print("警告：无法将总页数文本转换为整数，假设只有 1 页。")
                        total_pages = 1
                    except IndexError:
                        print("警告：未找到分页按钮，假设只有 1 页。")
                        total_pages = 1
                else:
                    print("警告：未找到分页相关的元素，假设只有 1 页。")
                    total_pages = 1

            elements = soup.find_all('div', class_=target_class)

            if elements:
                print(f"在第 {current_page} 页找到了 {len(elements)} 个匹配的元素：")
                for element in elements:
                    product_info = {
                        '产品名称': "N/A",
                        '原价': "N/A",
                        '现价': "N/A",
                        '单位价格': "N/A"
                    }
                    title_element = element.find("h2", class_="product__title")
                    was_price_element = element.find("span", class_="price__was")
                    now_price_element = element.find("span", class_="price__value")
                    unit_price_element = element.find("div", class_="price__calculation_method")

                    if title_element and title_element.contents:
                        product_info['产品名称'] = title_element.contents[0]
                    if was_price_element and was_price_element.contents:
                        product_info['原价'] = was_price_element.contents[0].replace(" | Was ", "")
                    if now_price_element and now_price_element.contents:
                        product_info['现价'] = now_price_element.contents[0]
                    if unit_price_element and unit_price_element.contents:
                        product_info['单位价格'] = unit_price_element.contents[0]

                    product_data.append(product_info)
            else:
                print(f"在第 {current_page} 页的 HTML 内容中未找到任何 class 为 '{target_class}' 的 div 元素。")
        else:
            print(f"错误：未能在第 {current_page} 页的 MHTML 文件中找到有效的 HTML 内容部分。")

        current_page += 1

except Exception as e:
    print(f"在处理页面或保存 MHTML 时出错: {e}")

finally:
    if driver:
        print("正在关闭 WebDriver...")
        driver.quit()
        print("WebDriver 已关闭。")
    else:
        print("WebDriver 未成功初始化，无需关闭。")

if product_data:
    print(f"\n正在将提取的 {len(product_data)} 条产品数据保存到 Excel 文件: {excel_file_name}")
    try:
        df = pd.DataFrame(product_data)
        df = df[['产品名称', '现价', '原价', '单位价格']]
        df.to_excel(excel_file_name, index=False, engine='openpyxl')
        print(f"Excel 文件 '{excel_file_name}' 保存成功。")
    except ImportError:
        print("错误: 需要安装 'pandas' 和 'openpyxl' 库来保存 Excel 文件。请运行: pip install pandas openpyxl")
    except Exception as ex:
        print(f"保存 Excel 文件时出错: {ex}")
else:
    print("没有提取到任何产品数据，未创建 Excel 文件。")

成功提取并解码第 1 页的 HTML 内容 (使用编码: utf-8)。
总页数已找到: 25
在第 1 页找到了 52 个匹配的元素：
成功提取并解码第 2 页的 HTML 内容 (使用编码: utf-8)。
在第 2 页找到了 48 个匹配的元素：
成功提取并解码第 3 页的 HTML 内容 (使用编码: utf-8)。
在第 3 页找到了 48 个匹配的元素：
成功提取并解码第 4 页的 HTML 内容 (使用编码: utf-8)。
在第 4 页找到了 48 个匹配的元素：
成功提取并解码第 5 页的 HTML 内容 (使用编码: utf-8)。
在第 5 页找到了 48 个匹配的元素：
成功提取并解码第 6 页的 HTML 内容 (使用编码: utf-8)。
在第 6 页找到了 48 个匹配的元素：
成功提取并解码第 7 页的 HTML 内容 (使用编码: utf-8)。
在第 7 页找到了 56 个匹配的元素：
成功提取并解码第 8 页的 HTML 内容 (使用编码: utf-8)。
在第 8 页找到了 48 个匹配的元素：
成功提取并解码第 9 页的 HTML 内容 (使用编码: utf-8)。
在第 9 页找到了 48 个匹配的元素：
成功提取并解码第 10 页的 HTML 内容 (使用编码: utf-8)。
在第 10 页找到了 48 个匹配的元素：
成功提取并解码第 11 页的 HTML 内容 (使用编码: utf-8)。
在第 11 页找到了 48 个匹配的元素：
成功提取并解码第 12 页的 HTML 内容 (使用编码: utf-8)。
在第 12 页找到了 48 个匹配的元素：
成功提取并解码第 13 页的 HTML 内容 (使用编码: utf-8)。
在第 13 页找到了 56 个匹配的元素：
成功提取并解码第 14 页的 HTML 内容 (使用编码: utf-8)。
在第 14 页找到了 48 个匹配的元素：
成功提取并解码第 15 页的 HTML 内容 (使用编码: utf-8)。
在第 15 页找到了 48 个匹配的元素：
成功提取并解码第 16 页的 HTML 内容 (使用编码: utf-8)。
在第 16 页找到了 48 个匹配的元素：
成功提取并解码第 17 页的 HTML 内容 (使用编码: utf-8)。
在第 17 页找到

In [ ]:
int(soup.find_all("span", class_="MuiButtonBase-root")[-1].get_text(strip=True))

'25'

In [1]:
!pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached regex-2024.11.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (40 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp312-cp312-macosx_11_0_arm64.whl (284 kB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [nltk]2/3 [nltk]b]


In [1]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import warnings
import tqdm
from multiprocessing import Pool, cpu_count # 导入 Pool 和 cpu_count
warnings.filterwarnings('ignore', category=UserWarning)

def _process_single_product(args):
    candidate_name, original_input_row, reference_data_tuples, reference_barcode_column, reference_desc_column = args
    candidate_tokens = word_tokenize(candidate_name)
    scores = []
    
    for ref_tokens, barcode, original_description in reference_data_tuples:
        bleu_score = sentence_bleu([ref_tokens], candidate_tokens, weights=(0.5, 0.5, 0, 0))
        scores.append({
            'bleu_score': bleu_score,
            'Barcode': barcode,
            'Description': original_description
        })
        
    top_match = sorted(scores, key=lambda x: x['bleu_score'], reverse=True)[0]
    result_row = original_input_row.to_dict() # 将原始行转换为字典
    result_row['最佳匹配_Barcode'] = top_match['Barcode']
    result_row['最佳匹配_Description'] = top_match['Description']
    
    return result_row

def find_top_bleu_matches(
    input_file_path: str,
    reference_file_path: str,
    input_column_name: str = '产品名称',
    reference_barcode_column: str = 'Barcode',
    reference_desc_column: str = 'Description'
) -> pd.DataFrame:
    try:
        input_df = pd.read_excel(input_file_path)
        reference_df = pd.read_excel(reference_file_path, dtype={reference_barcode_column: str})

    except FileNotFoundError as e:
        return pd.DataFrame()

    reference_df[reference_desc_column] = reference_df[reference_desc_column].astype(str).str.lower()
    input_df[input_column_name] = input_df[input_column_name].astype(str).str.lower() # 确保用于匹配的列是小写

    reference_data_tuples = list(zip(
        reference_df[reference_desc_column].apply(word_tokenize),
        reference_df[reference_barcode_column],
        reference_df[reference_desc_column]
    ))
    
    all_results = []

    # 4. 准备并行处理的输入参数列表
    # 现在每个任务除了产品名称，还包含原始的行数据
    tasks = [
        (row[input_column_name], row, reference_data_tuples, reference_barcode_column, reference_desc_column)
        for index, row in input_df.iterrows()
    ]

    # 5. 使用 multiprocessing.Pool 进行并行处理
    print(f"开始使用 {cpu_count()} 个核心进行并行处理...")
    with Pool(cpu_count()) as pool:
        for result_row in tqdm.tqdm(pool.imap(_process_single_product, tasks), total=len(tasks), desc="处理产品名称"):
            all_results.append(result_row)
    
    # 8. 将最终结果转换为DataFrame
    final_df = pd.DataFrame(all_results)

    # 9. 确保 Barcode 列在写入Excel前是字符串类型，以防止科学计数法
    if '最佳匹配_Barcode' in final_df.columns:
        final_df['最佳匹配_Barcode'] = final_df['最佳匹配_Barcode'].astype(str)

    return final_df


# --- 使用示例 ---
if __name__ == '__main__':
    # 定义您的文件路径和列名
    INPUT_FILE = 'coles_data.xlsx' # 假设这个文件包含了 '原价', '现价', '单位价格' 等列
    DATABASE_FILE = 'Item Barcode.xlsx'
    
    # 调用函数
    results_df = find_top_bleu_matches(
        input_file_path=INPUT_FILE,
        reference_file_path=DATABASE_FILE
    )
    
    # 打印结果并保存到新的Excel文件
    if not results_df.empty:
        print("\n--- 最终匹配结果 ---")
        print(results_df)
        output_file_name = 'coles_matches.xlsx' # 更名以区分
        results_df.to_excel(output_file_name, index=False)
        print(f"\n结果已成功保存到 '{output_file_name}'")

开始使用 10 个核心进行并行处理...


处理产品名称:   0%|          | 0/1479 [00:00<?, ?it/s]Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/crawler/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/miniconda3/envs/crawler/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/envs/crawler/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/miniconda3/envs/crawler/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute '_process_single_product' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/crawler/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "

KeyboardInterrupt: 

In [6]:
INPUT_FILE = 'woolworths_data.xlsx'
DATABASE_FILE = 'Item Barcode.xlsx'

# 调用函数
results_df = find_top_bleu_matches(
    input_file_path=INPUT_FILE,
    reference_file_path=DATABASE_FILE
)

# 打印结果并保存到新的Excel文件
if not results_df.empty:
    print("\n--- 最终匹配结果 ---")
    print(results_df)
    output_file_name = 'woolworths_matches.xlsx'
    results_df.to_excel(output_file_name, index=False)
    print(f"\n结果已成功保存到 '{output_file_name}'")

成功读取Excel文件。
已将参考描述列和输入产品名称列转换为小写。


处理产品名称: 100%|██████████| 2650/2650 [25:15<00:00,  1.75it/s]


--- 最终匹配结果 ---
                                                输入的产品名称   最佳匹配_Barcode  \
0        mr kipling cake angel slices snack pack 6 pack  9349673004450   
1     mount franklin lightly sparkling water lime mu...  9300624017622   
2     dynamo professional laundry liquid odour elimi...  9350376003619   
3      john west chunky tuna tempter in springwater 95g  9300462343716   
4                  vanish gold stain remover powder 2kg  9300605122994   
...                                                 ...            ...   
2645  shangri-la linen cashmere touch flannelette fi...  9338972002656   
2646  shangri-la linen cashmere touch flannelette fi...  9310460036565   
2647  shangri-la linen cashmere touch flannelette fi...  9338972002656   
2648  shangri-la linen cashmere touch flannelette fi...  9338972002649   
2649  shangri-la linen cashmere touch flannelette fi...  9333441008146   

                    最佳匹配_Description  
0          mr chen's easy snack buns  
1      mf sparkli

In [7]:
import gradio as gr
import pandas as pd

# 用于存储已加载的数据帧
coles_df = None
woolworths_df = None

def load_data(coles_file, woolworths_file):
    """加载 Coles 和 Woolworths 的匹配数据"""
    global coles_df, woolworths_df
    try:
        if coles_file:
            coles_df = pd.read_excel(coles_file.name)
            # 确保 '输入的产品名称' 列是字符串类型，以便后续比较
            coles_df['输入的产品名称'] = coles_df['输入的产品名称'].astype(str).str.lower()
            print("Coles 数据加载成功！")
        if woolworths_file:
            woolworths_df = pd.read_excel(woolworths_file.name)
            # 确保 '输入的产品名称' 列是字符串类型，以便后续比较
            woolworths_df['输入的产品名称'] = woolworths_df['输入的产品名称'].astype(str).str.lower()
            print("Woolworths 数据加载成功！")
        return "文件已成功加载！现在可以输入商品名称进行查询。", ""
    except Exception as e:
        return f"加载文件时出错：{e}", ""

def compare_prices(product_name):
    """根据商品名称比较 Coles 和 Woolworths 的匹配结果"""
    if coles_df is None and woolworths_df is None:
        return "请先上传 Coles 和 Woolworths 的匹配文件。", None, None

    # 将输入的商品名称转换为小写，进行不区分大小写的匹配
    search_name = str(product_name).lower()

    coles_result = "Coles: 未找到匹配项。"
    woolworths_result = "Woolworths: 未找到匹配项。"

    coles_barcode_df = pd.DataFrame()
    woolworths_barcode_df = pd.DataFrame()

    if coles_df is not None:
        # 使用 str.contains 进行模糊匹配，更加灵活
        coles_match = coles_df[coles_df['输入的产品名称'].str.contains(search_name, na=False)]
        if not coles_match.empty:
            # 确保 Barcode 是字符串
            coles_match['最佳匹配_Barcode'] = coles_match['最佳匹配_Barcode'].astype(str)
            coles_result = f"Coles - 找到 {len(coles_match)} 个匹配项。"
            coles_barcode_df = coles_match[['输入的产品名称', '最佳匹配_Barcode', '最佳匹配_Description']]

    if woolworths_df is not None:
        # 使用 str.contains 进行模糊匹配
        woolworths_match = woolworths_df[woolworths_df['输入的产品名称'].str.contains(search_name, na=False)]
        if not woolworths_match.empty:
            # 确保 Barcode 是字符串
            woolworths_match['最佳匹配_Barcode'] = woolworths_match['最佳匹配_Barcode'].astype(str)
            woolworths_result = f"Woolworths - 找到 {len(woolworths_match)} 个匹配项。"
            woolworths_barcode_df = woolworths_match[['输入的产品名称', '最佳匹配_Barcode', '最佳匹配_Description']]
    
    status_message = f"{coles_result}\n{woolworths_result}"
    return status_message, coles_barcode_df, woolworths_barcode_df

# Gradio 界面定义
with gr.Blocks() as demo:
    gr.Markdown("# 商品比价工具")
    gr.Markdown("请上传 Coles 和 Woolworths 的匹配结果 Excel 文件，然后输入商品名称进行比价。")

    with gr.Row():
        coles_file_input = gr.File(label="上传 Coles 匹配文件 (coles_matches.xlsx)", file_types=[".xlsx"])
        woolworths_file_input = gr.File(label="上传 Woolworths 匹配文件 (woolworths_matches.xlsx)", file_types=[".xlsx"])
        load_button = gr.Button("加载文件")

    status_output = gr.Textbox(label="文件加载状态", interactive=False)

    with gr.Row():
        product_name_input = gr.Textbox(label="输入要查询的商品名称", placeholder="例如：牛奶")
        compare_button = gr.Button("开始比价")
    
    compare_status_output = gr.Textbox(label="比价结果状态", interactive=False)
    coles_output = gr.DataFrame(label="Coles 匹配结果")
    woolworths_output = gr.DataFrame(label="Woolworths 匹配结果")

    # 绑定事件
    load_button.click(
        fn=load_data,
        inputs=[coles_file_input, woolworths_file_input],
        outputs=[status_output, product_name_input] # 清空商品名称输入框
    )
    compare_button.click(
        fn=compare_prices,
        inputs=product_name_input,
        outputs=[compare_status_output, coles_output, woolworths_output]
    )

# 启动 Gradio 应用
if __name__ == "__main__":
    demo.launch() # 在本地启动，或使用 launch(share=True) 创建一个可分享的链接

/opt/miniconda3/envs/crawler/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
